In [98]:
import time
from datetime import date
import pandas as pd
import os
import glob
import pycytominer
import sys
import CBE_utils as CBE
import matplotlib.pyplot as plt
import seaborn as sns

In [99]:
import importlib
importlib.reload(CBE)

<module 'CBE_utils' from '/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis/notebooks/CBE_utils.py'>

In [100]:
input_path = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis/output_files/"
output_path = "/home/schmiedc/FMP_Docs/Projects/ECBL_Project/QualityControl_analysis/result_files/"

In [101]:
# data_HepG2 = pd.read_csv(input_path + "2023-04-11_Bioactives_HepG2_raw.csv")
pattern = "[A-Z][0-9][0-9][0-9][0-9]_R[1-4].csv"
files = glob.glob(input_path + os.sep + '*' + os.sep + pattern, recursive=True)

In [102]:
for file in files:
    
    filename = os.path.basename(file)
    
    try:
        
        csvreader = pd.read_csv(file)
        row_count = csvreader.shape[0]
        
        print(f"File: {filename} has {row_count} rows")
            
    except Exception as e:
        
        print(f"Error reading file {filename}: {e}")

File: C1160_R2.csv has 384 rows
File: C1160_R3.csv has 384 rows
File: C1091_R1.csv has 384 rows
File: C1092_R1.csv has 384 rows
File: C1089_R1.csv has 384 rows
File: C1085_R1.csv has 384 rows
File: C1093_R1.csv has 384 rows
File: C1158_R4.csv has 384 rows
File: C1158_R2.csv has 384 rows
File: C1157_R4.csv has 384 rows
File: C1159_R2.csv has 384 rows
File: C1160_R1.csv has 384 rows
File: C1157_R3.csv has 384 rows
File: C1086_R1.csv has 384 rows
File: C1160_R4.csv has 384 rows
File: C1087_R1.csv has 384 rows
File: C1157_R1.csv has 384 rows
File: C1088_R1.csv has 384 rows
File: C1159_R3.csv has 384 rows
File: C1090_R1.csv has 384 rows
File: C1084_R1.csv has 384 rows


In [103]:
### load file
dataframe = pd.read_csv(files[-1])
dataframe.head()

,Metadata_Batch,Metadata_Plate,Metadata_Well_randomized,Metadata_Object_Count,Nuc_AreaShape_Area,Nuc_AreaShape_BoundingBoxArea,Nuc_AreaShape_BoundingBoxMaximum_X,Nuc_AreaShape_BoundingBoxMaximum_Y,Nuc_AreaShape_BoundingBoxMinimum_X,Nuc_AreaShape_BoundingBoxMinimum_Y,...,Metadata_plate_map_name,Metadata_user,Metadata_source,Metadata_instrument,Metadata_imaging_date,Metadata_imaging_time,Metadata_imaging_timezone,Metadata_picklist_name,Metadata_Well,Metadata_RoughID
0,HepG2_10uM,C1084R1,A01,1924,497.0,675.0,515.5,537.5,488.5,507.0,...,C1084_R1,Christopher,FMP,Phenix,2024-06-05,13:34:51,200,Picklist_Replicate1.csv,O02,EOS_cpd
1,HepG2_10uM,C1084R1,A02,1668,499.0,676.0,632.5,486.5,602.5,458.0,...,C1084_R1,Christopher,FMP,Phenix,2024-06-05,13:34:51,200,Picklist_Replicate1.csv,K17,EOS_cpd
2,HepG2_10uM,C1084R1,A03,2265,499.0,690.0,532.0,540.0,505.0,509.0,...,C1084_R1,Christopher,FMP,Phenix,2024-06-05,13:34:51,200,Picklist_Replicate1.csv,I19,EOS_cpd
3,HepG2_10uM,C1084R1,A04,1888,520.0,713.0,552.0,572.5,522.0,541.0,...,C1084_R1,Christopher,FMP,Phenix,2024-06-05,13:34:51,200,Picklist_Replicate1.csv,B09,EOS_cpd
4,HepG2_10uM,C1084R1,A05,2336,503.0,690.0,563.5,527.5,534.0,498.0,...,C1084_R1,Christopher,FMP,Phenix,2024-06-05,13:34:51,200,Picklist_Replicate1.csv,N10,EOS_cpd


In [104]:
### helper functions extracts metadata columns and define feature columns
features = CBE.get_feature_vector(dataframe)
metadata_dataframe = set(dataframe.columns) - set(features)

### convert sets to list
features_list = list(features)
metadata_list = list(metadata_dataframe)

print("Total number of features:", len(features_list) )

Total number of features: 2977


In [105]:
# TODO: Adjust to future pycytominer version
normalizer = "mad_robustize" 

dataframe_normalized = pycytominer.normalize(
    
    profiles = dataframe,
    features = features,
    meta_features = metadata_dataframe,
    method = normalizer, ### Method to normalize the data
    samples = "Metadata_RoughID == 'DMSO'" # normalization performed on neg. controls
)

/home/schmiedc/miniconda3/envs/paint/lib/python3.9/site-packages/pycytominer/normalize.py:151: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  meta_df = profiles.loc[:, meta_features]


In [106]:
# TODO: Feature selection options
dataframe_normalized

,Metadata_imaging_timezone,Metadata_source,Metadata_plate_map_name,Metadata_imaging_date,Metadata_Well_randomized,Metadata_imaging_time,Metadata_RoughID,Metadata_picklist_name,Metadata_plate_name,Metadata_user,...,Cyto_Texture_Variance_Mito_10_02_256,Cyto_Texture_Variance_Mito_10_03_256,Cyto_Texture_Variance_Mito_3_00_256,Cyto_Texture_Variance_Mito_3_01_256,Cyto_Texture_Variance_Mito_3_02_256,Cyto_Texture_Variance_Mito_3_03_256,Cyto_Texture_Variance_Mito_5_00_256,Cyto_Texture_Variance_Mito_5_01_256,Cyto_Texture_Variance_Mito_5_02_256,Cyto_Texture_Variance_Mito_5_03_256
0,200,FMP,C1084_R1,2024-06-05,A01,13:34:51,EOS_cpd,Picklist_Replicate1.csv,C1084,Christopher,...,-4.601744,-4.151888,-4.502443,-4.629765,-4.580026,-4.334689,-4.658107,-4.361580,-4.324040,-4.541473
1,200,FMP,C1084_R1,2024-06-05,A02,13:34:51,EOS_cpd,Picklist_Replicate1.csv,C1084,Christopher,...,-4.800760,-4.299846,-4.665011,-4.804625,-4.732813,-4.476484,-4.822309,-4.530918,-4.476955,-4.712999
2,200,FMP,C1084_R1,2024-06-05,A03,13:34:51,EOS_cpd,Picklist_Replicate1.csv,C1084,Christopher,...,-4.239203,-3.796197,-4.119379,-4.237940,-4.192466,-3.964202,-4.264985,-3.994418,-3.974347,-4.167530
3,200,FMP,C1084_R1,2024-06-05,A04,13:34:51,EOS_cpd,Picklist_Replicate1.csv,C1084,Christopher,...,-4.694948,-4.218304,-4.609560,-4.727400,-4.667283,-4.416973,-4.756894,-4.435004,-4.406495,-4.617876
4,200,FMP,C1084_R1,2024-06-05,A05,13:34:51,EOS_cpd,Picklist_Replicate1.csv,C1084,Christopher,...,-4.569905,-4.084159,-4.441431,-4.567102,-4.501193,-4.275153,-4.591921,-4.283667,-4.266774,-4.475196
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,200,FMP,C1084_R1,2024-06-05,P20,13:34:51,EOS_cpd,Picklist_Replicate1.csv,C1084,Christopher,...,-1.705553,-1.465142,-1.756928,-1.781003,-1.769662,-1.685991,-1.812584,-1.688029,-1.691946,-1.745524
380,200,FMP,C1084_R1,2024-06-05,P21,13:34:51,EOS_cpd,Picklist_Replicate1.csv,C1084,Christopher,...,-1.972032,-1.857823,-1.979235,-1.996004,-1.968246,-1.875609,-2.028821,-1.828788,-1.878335,-1.971089
381,200,FMP,C1084_R1,2024-06-05,P22,13:34:51,EOS_cpd,Picklist_Replicate1.csv,C1084,Christopher,...,-4.620253,-4.135442,-4.489013,-4.615604,-4.559774,-4.306719,-4.632069,-4.340768,-4.312936,-4.530601
382,200,FMP,C1084_R1,2024-06-05,P23,13:34:51,EOS_cpd,Picklist_Replicate1.csv,C1084,Christopher,...,-4.699753,-4.197055,-4.581877,-4.704640,-4.640506,-4.399084,-4.732814,-4.410193,-4.385307,-4.619376


# Correlation based comparison of pos. ctrl wells

In [107]:
# perfectly correlated with correlation coefficient equal to 1 has zero distance 
# perfectly negatively correlated with correlation coefficient equal to -1 has maximal distance at 2
# inverse of correlation and only looks at the angle/similarity among patterns (sort of like normalization). 
# Correlational distance goes from 0 - 2, with 0 being PERFECT correlation, 1 being no correlation, and 2 being PERFECT ANTICORRELATION. 

from scipy.spatial.distance import correlation

In [108]:
# Filter for pos. control tetrandrine
# Metadata_RoughID = Tetrandrine
# Should be C24 and D24
tetrandrine_dataframe = dataframe_normalized.loc[dataframe['Metadata_RoughID'] == 'Tetrandrine']

# get features for first and second tet well
tetrandrine_well_1_features = tetrandrine_dataframe.iloc[0][features].values
tetrandrine_well_2_features = tetrandrine_dataframe.iloc[1][features].values
tetrandrine_median_features = tetrandrine_dataframe[features].median(axis=0)

corr_tetrandrine = correlation(tetrandrine_well_1_features, tetrandrine_well_2_features, centered = True)
corr_tetrandrine

0.013654087616855493

In [109]:
# Filter pos. ctrl Nocodazole
# Metadata_RoughID = Nocodazole
# Should be A24 and B24
nocodazole_dataframe = dataframe_normalized.loc[dataframe['Metadata_RoughID'] == 'Nocodazole']

# get features for first and second noco well
nocodazole_well_1_features = nocodazole_dataframe.iloc[0][features].values
nocodazole_well_2_features = nocodazole_dataframe.iloc[1][features].values
nocodazole_median_features = nocodazole_dataframe[features].median(axis=0)

corr_nocodazole = correlation(nocodazole_well_1_features, nocodazole_well_2_features, centered = True)
corr_nocodazole

0.027917518926815488

In [110]:
corr_pos_ctrl_1 = correlation(tetrandrine_well_1_features, nocodazole_well_1_features, centered = True)
corr_pos_ctrl_1

0.5449333682351922

In [111]:
corr_pos_ctrl_2 = correlation(tetrandrine_well_1_features, nocodazole_well_2_features, centered = True)
corr_pos_ctrl_2

0.5699360972313822

In [112]:
corr_pos_ctrl_3 = correlation(tetrandrine_well_2_features, nocodazole_well_2_features, centered = True)
corr_pos_ctrl_3

0.5424227463881108

In [113]:
corr_pos_ctrl_4 = correlation(tetrandrine_well_2_features, nocodazole_well_1_features, centered = True)
corr_pos_ctrl_4

0.5218925934347285

In [114]:
corr_pos_ctrl_median = correlation(tetrandrine_median_features, nocodazole_median_features, centered = True)
corr_pos_ctrl_median

0.5402529182809406

In [115]:
# Filter pos. ctrl Nocodazole
# Metadata_RoughID = Nocodazole
# Should be A24 and B24
dmso_dataframe = dataframe_normalized.loc[dataframe['Metadata_RoughID'] == 'DMSO']
dmso_median_features = dmso_dataframe[features].median(axis=0)

corr_noco_dmso = correlation(nocodazole_median_features, dmso_median_features, centered = True)
corr_noco_dmso

1.0002360393902132

In [116]:
corr_tet_dmso = correlation(tetrandrine_median_features, dmso_median_features, centered = True)
corr_tet_dmso

1.0002060870247629

## Other correlation based comparison

In [117]:
import numpy as np

In [118]:
def feature_corr(df1, df2):
    
    corr = np.corrcoef(df1.astype(float), df2.astype(float))
    np.fill_diagonal(corr, np.nan)
    
    return np.nanmedian(corr)

In [119]:
feature_corr(tetrandrine_median_features, nocodazole_median_features)

0.4597470817190612

In [120]:
feature_corr(tetrandrine_well_1_features, tetrandrine_well_2_features)

0.9863459123830671

In [121]:
feature_corr(nocodazole_well_1_features, nocodazole_well_2_features)

0.9720824810732367

In [122]:
# Overall seems to yield useful values
# We can compute consensus values also then
# Test to see what happens if it fails i.e. wrong platelayout is used
# Thresholds could be: <0.3 for corr
# >0.3 for non correlation
# Oberservation from real world data will help
# Also again feature selection will make this more robust
# An alternative could be to use DMSO independent normalization
# At least for the detection of Plate layout issue

# Comparison with other neg. and pos. controls

In [123]:
# TODO: Correlation of pos. & neg. controls over time
# TODO: First take any plate for comparison
# TODO: Implement consensus of first plates of specific source
# TODO: Then with an external control from the bioactives

In [147]:
### load file
other_dataframe = pd.read_csv(files[2])

### helper functions extracts metadata columns and define feature columns
other_features = CBE.get_feature_vector(other_dataframe)
other_metadata_dataframe = set(other_dataframe.columns) - set(features)

normalizer = "mad_robustize" 

# TODO: Adjust to future pycytominer version
other_dataframe_normalized = pycytominer.normalize(
    
    profiles = other_dataframe,
    features = other_features,
    meta_features = other_metadata_dataframe,
    method = normalizer, ### Method to normalize the data
    samples = "Metadata_RoughID == 'DMSO'" # normalization performed on neg. controls
)

/home/schmiedc/miniconda3/envs/paint/lib/python3.9/site-packages/pycytominer/normalize.py:151: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  meta_df = profiles.loc[:, meta_features]


In [148]:
# Filter for pos. control tetrandrine
# Metadata_RoughID = Tetrandrine
# Should be C24 and D24
other_tetrandrine_dataframe = other_dataframe_normalized.loc[dataframe['Metadata_RoughID'] == 'Tetrandrine']

# get features for first and second tet well
other_tetrandrine_median_features = other_tetrandrine_dataframe[features].median(axis=0)

correlation(other_tetrandrine_median_features, tetrandrine_median_features, centered = True)

0.685135273510096

In [149]:
# Filter pos. ctrl Nocodazole
# Metadata_RoughID = Nocodazole
# Should be A24 and B24
other_nocodazole_dataframe = other_dataframe_normalized.loc[dataframe['Metadata_RoughID'] == 'Nocodazole']

# get features for first and second noco well
other_nocodazole_median_features = other_nocodazole_dataframe[features].median(axis=0)
correlation(nocodazole_median_features, other_nocodazole_median_features, centered = True)

0.0958805673067663

In [150]:
correlation(other_tetrandrine_median_features, other_nocodazole_median_features)

0.3500988296480524

In [151]:
# Filter pos. ctrl Nocodazole
# Metadata_RoughID = Nocodazole
# Should be A24 and B24
other_dmso_dataframe = other_dataframe_normalized.loc[dataframe['Metadata_RoughID'] == 'DMSO']
other_dmso_median_features = dmso_dataframe[features].median(axis=0)

correlation(other_dmso_median_features, dmso_median_features, centered = True)

0.0

In [152]:
feature_corr(other_tetrandrine_median_features, tetrandrine_median_features)

0.3148647264898983

In [153]:
feature_corr(other_nocodazole_median_features, nocodazole_median_features)

0.904119432693222

In [154]:
feature_corr(other_dmso_median_features, dmso_median_features)

1.0

In [155]:
# TODO: Automate over all available plates
# TODO: Record as table with date, plate_names, source, user and other information

# Compute metrics over replicates

In [ ]:
# TODO: If all replicates exist:
# TODO: Feature reduction per plate
# TODO: Compute number of toxic compounds
# TODO: Compute raw % replication
# TODO: Compute compound activity based on DMSO
# TODO: Compute % replication after compound filters

# Next Layer of implementation

In [ ]:
# TODO: QC visualization: per plate, per batch, per source

In [ ]:
# TODO: Setup plate layout test to check if faulty plate layout will get detected
# TODO: Check if alternative normalization method that is independent of the layout

In [ ]:
# TODO: Implement Correlation of pos. & neg. controls over time and track over time

In [ ]:
# Better automation with data that trickles in